In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat

### Open gadm0_precalculated
Geojson downloaded from AGOL

In [2]:
gadm_precalc = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/gadm0_precalculated_20220224_backup2.geojson')


In [3]:
gadm_precalc.head()

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,majority_land_cover_climate_reg,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,geometry
0,ABW,Aruba,1,1.819384e+02,"[{""SliceNumber"":1819,""per_global"":0.6,""per_aoi...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":353,""per_global"":0.01,""per_aoi...",20.609300,115656.10,335.0,Shrubland,Tropical Dry,None,24.66,None,6.39,5,1,"POLYGON ((-69.97820 12.46986, -70.02847 12.503..."
1,AFG,Afghanistan,2,6.438575e+05,"[{""SliceNumber"":32,""per_global"":100.0,""per_aoi...","[{""SliceNumber"":26,""per_global"":1.44,""per_aoi""...","[{""SliceNumber"":167,""per_global"":11.15,""per_ao...","[{""SliceNumber"":1,""per_global"":28.71,""per_aoi""...",3.785337,30389700.00,142.0,Grassland,Cool Temperate Dry,6.48,18.49,62.15,0.07,3,2,"POLYGON ((68.52644 31.75435, 68.53852 31.75457..."
2,AGO,Angola,3,1.247422e+06,"[{""SliceNumber"":2,""per_global"":0.59,""per_aoi"":...","[{""SliceNumber"":26,""per_global"":4.67,""per_aoi""...","[{""SliceNumber"":28,""per_global"":2.22,""per_aoi""...","[{""SliceNumber"":40,""per_global"":26.68,""per_aoi...",6.679349,36074940.00,268.0,Forest,Sub Tropical Moist,0.07,4.19,67.89,0.08,2,3,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -..."
3,AIA,Anguilla,4,8.330331e+01,"[{""SliceNumber"":1819,""per_global"":0.23,""per_ao...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":316,""per_global"":25.32,""per_ao...",2.880000,13601.96,393.0,Grassland,Tropical Moist,None,NaN,None,NaN,5,4,"MULTIPOLYGON (((-63.42375 18.58903, -63.42847 ..."
4,ALA,Åland,5,1.506261e+03,"[{""SliceNumber"":963,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":116,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":129,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":2500,""per_global"":0.01,""per_ao...",1.449518,29158.96,111.0,Forest,Cool Temperate Moist,None,50.27,None,NaN,5,5,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59..."


In [4]:
len(gadm_precalc)

255

In [5]:
type(gadm_precalc)

geopandas.geodataframe.GeoDataFrame

### Open gadm version 4.0

In [6]:
gadm40 = gpd.read_file('/Users/sofia/Documents/HE_Data/gadm/gadm404-shp/gadm404.shp')


In [10]:
len(gadm40)

348904

### Create new table with gadm40 in which we only have GID_0 and NAME_0

In [7]:
gadm40_GID = gadm40[['GID_0', 'NAME_0']]
gadm40_GID = gadm40_GID.groupby('GID_0')
gadm40_GID = gadm40_GID.first()
gadm40_GID = gadm40_GID.reset_index()
gadm40_GID

,GID_0,NAME_0
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland
...,...,...
258,Z09,Sang
259,ZAF,South Africa
260,ZMB,Zambia
261,ZNC,Northern Cyprus


In [8]:
gadm40_GID = gadm40_GID.rename(columns={'GID_0':'GID', 'NAME_0':'NAME'})
gadm40_GID

,GID,NAME
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALA,Åland
...,...,...
258,Z09,Sang
259,ZAF,South Africa
260,ZMB,Zambia
261,ZNC,Northern Cyprus


In [9]:
gadm = pd.merge(gadm_precalc, gadm40_GID, how='left', left_on='GID_0', right_on='GID')
gadm.head()

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,geometry,GID,NAME
0,ABW,Aruba,1,1.819384e+02,"[{""SliceNumber"":1819,""per_global"":0.6,""per_aoi...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":353,""per_global"":0.01,""per_aoi...",20.609300,115656.10,...,Tropical Dry,None,24.66,None,6.39,5,1,"POLYGON ((-69.97820 12.46986, -70.02847 12.503...",ABW,Aruba
1,AFG,Afghanistan,2,6.438575e+05,"[{""SliceNumber"":32,""per_global"":100.0,""per_aoi...","[{""SliceNumber"":26,""per_global"":1.44,""per_aoi""...","[{""SliceNumber"":167,""per_global"":11.15,""per_ao...","[{""SliceNumber"":1,""per_global"":28.71,""per_aoi""...",3.785337,30389700.00,...,Cool Temperate Dry,6.48,18.49,62.15,0.07,3,2,"POLYGON ((68.52644 31.75435, 68.53852 31.75457...",AFG,Afghanistan
2,AGO,Angola,3,1.247422e+06,"[{""SliceNumber"":2,""per_global"":0.59,""per_aoi"":...","[{""SliceNumber"":26,""per_global"":4.67,""per_aoi""...","[{""SliceNumber"":28,""per_global"":2.22,""per_aoi""...","[{""SliceNumber"":40,""per_global"":26.68,""per_aoi...",6.679349,36074940.00,...,Sub Tropical Moist,0.07,4.19,67.89,0.08,2,3,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -...",AGO,Angola
3,AIA,Anguilla,4,8.330331e+01,"[{""SliceNumber"":1819,""per_global"":0.23,""per_ao...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":316,""per_global"":25.32,""per_ao...",2.880000,13601.96,...,Tropical Moist,None,NaN,None,NaN,5,4,"MULTIPOLYGON (((-63.42375 18.58903, -63.42847 ...",AIA,Anguilla
4,ALA,Åland,5,1.506261e+03,"[{""SliceNumber"":963,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":116,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":129,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":2500,""per_global"":0.01,""per_ao...",1.449518,29158.96,...,Cool Temperate Moist,None,50.27,None,NaN,5,5,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59...",ALA,Åland


In [10]:
gadm[['GID_0', 'NAME_0', 'GID', 'NAME']]

,GID_0,NAME_0,GID,NAME
0,ABW,Aruba,ABW,Aruba
1,AFG,Afghanistan,AFG,Afghanistan
2,AGO,Angola,AGO,Angola
3,AIA,Anguilla,AIA,Anguilla
4,ALA,Åland,ALA,Åland
...,...,...,...,...
250,XSP,Spratly Islands,XSP,Spratly Islands
251,YEM,Yemen,YEM,Yemen
252,ZAF,South Africa,ZAF,South Africa
253,ZMB,Zambia,ZMB,Zambia


In [11]:
gadm2 = gadm[gadm.NAME_0!=gadm.NAME]
gadm2[['NAME_0', 'NAME']]

,NAME_0,NAME
47,Republic of Congo,Republic of the Congo
51,Cape Verde,Cabo Verde
58,Czech Republic,Czechia
96,Hong Kong,NaN
135,Macao,NaN
144,Macedonia,North Macedonia
184,Palestina,Palestine
187,Reunion,Réunion
196,Saint Helena,"Saint Helena, Ascension and Tristan da Cunha"
248,Northern Cyprus,NaN


In [12]:
gadm.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'geometry', 'GID', 'NAME'],
      dtype='object')

In [13]:
gadm.NAME.fillna(gadm.NAME_0, inplace=True)


In [14]:
gadm[gadm.NAME_0 != gadm.NAME]

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,geometry,GID,NAME
47,COG,Republic of Congo,48,341732.300,"[{""SliceNumber"":2,""per_global"":24.07,""per_aoi""...","[{""SliceNumber"":31,""per_global"":11.95,""per_aoi...","[{""SliceNumber"":70,""per_global"":3.16,""per_aoi""...","[{""SliceNumber"":160,""per_global"":1.38,""per_aoi...",36.486540,4.039875e+06,...,Tropical Moist,None,1.62,48.24,0.33,4,48,"MULTIPOLYGON (((11.69764 -4.46764, 11.70014 -4...",COG,Republic of the Congo
51,CPV,Cape Verde,52,4090.953,"[{""SliceNumber"":252,""per_global"":0.05,""per_aoi...","[{""SliceNumber"":102,""per_global"":98.58,""per_ao...","[{""SliceNumber"":2335,""per_global"":0.04,""per_ao...","[{""SliceNumber"":2171,""per_global"":96.43,""per_a...",3.010442,5.312174e+05,...,Sub Tropical Dry,None,10.42,None,NaN,5,52,"MULTIPOLYGON (((-24.68681 14.81264, -24.70236 ...",CPV,Cabo Verde
58,CZE,Czech Republic,59,78835.730,"[{""SliceNumber"":815,""per_global"":2.47,""per_aoi...","[{""SliceNumber"":42,""per_global"":0.22,""per_aoi""...","[{""SliceNumber"":129,""per_global"":0.18,""per_aoi...","[{""SliceNumber"":573,""per_global"":0.2,""per_aoi""...",22.839670,1.084056e+07,...,Cool Temperate Moist,0.87,85.86,0,2.67,5,59,"POLYGON ((14.63085 48.60770, 14.62451 48.60737...",CZE,Czechia
144,MKD,Macedonia,145,24825.140,"[{""SliceNumber"":819,""per_global"":2.19,""per_aoi...","[{""SliceNumber"":29,""per_global"":0.32,""per_aoi""...","[{""SliceNumber"":253,""per_global"":0.22,""per_aoi...","[{""SliceNumber"":7,""per_global"":0.92,""per_aoi"":...",8.061651,2.098622e+06,...,Cool Temperate Moist,8.66,53.60,9.65,1.39,5,145,"POLYGON ((22.74947 41.18259, 22.73658 41.16040...",MKD,North Macedonia
184,PSE,Palestina,185,6220.036,"[{""SliceNumber"":947,""per_global"":0.19,""per_aoi...","[{""SliceNumber"":97,""per_global"":0.0,""per_aoi"":...","[{""SliceNumber"":33,""per_global"":0.17,""per_aoi""...","[{""SliceNumber"":10,""per_global"":15.45,""per_aoi...",9.424097,5.742385e+06,...,Warm Temperate Dry,7.61,43.47,1.13,7.60,5,185,"MULTIPOLYGON (((34.26801 31.22361, 34.22903 31...",PSE,Palestine
187,REU,Reunion,188,2512.652,"[{""SliceNumber"":238,""per_global"":0.03,""per_aoi...","[{""SliceNumber"":92,""per_global"":0.03,""per_aoi""...","[{""SliceNumber"":3218,""per_global"":97.72,""per_a...","[{""SliceNumber"":3924,""per_global"":1.2,""per_aoi...",64.881290,9.381416e+05,...,Warm Temperate Moist,6.65,26.22,None,4.80,5,188,"POLYGON ((55.58816 -21.37458, 55.55042 -21.375...",REU,Réunion
196,SHN,Saint Helena,197,411.226,"[{""SliceNumber"":5877,""per_global"":0.02,""per_ao...","[{""SliceNumber"":92,""per_global"":0.0,""per_aoi"":...","[{""SliceNumber"":2955,""per_global"":0.01,""per_ao...",None,4.232432,3.616648e+03,...,Warm Temperate Moist,None,NaN,None,NaN,5,197,"MULTIPOLYGON (((-9.91888 -40.37096, -9.91825 -...",SHN,"Saint Helena, Ascension and Tristan da Cunha"


In [15]:
gadm.NAME_0 = gadm.NAME
gadm = gadm.drop(columns={'NAME', 'GID'})

In [16]:
gadm.NAME_0[gadm.GID_0=='CZE']

58    Czechia
Name: NAME_0, dtype: object

In [17]:
gadm_precalc.NAME_0[gadm_precalc.GID_0=='CZE']

58    Czech Republic
Name: NAME_0, dtype: object

In [18]:
gadm.NAME_0[gadm.GID_0=='HKG']

96    Hong Kong
Name: NAME_0, dtype: object

In [28]:
gadm[gadm.NAME_0 == 'Swaziland']

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,majority_land_cover_climate_reg,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,geometry
211,SWZ,Swaziland,212,17383.03,"[{""SliceNumber"":33,""per_global"":8.47,""per_aoi""...","[{""SliceNumber"":26,""per_global"":0.08,""per_aoi""...","[{""SliceNumber"":28,""per_global"":0.21,""per_aoi""...","[{""SliceNumber"":40,""per_global"":0.21,""per_aoi""...",4.35874,1097640.0,272.0,Cropland,Sub Tropical Dry,6.14,3.09,84.38,1.5,5,212,"POLYGON ((31.55598 -27.31561, 31.52677 -27.315..."


In [19]:
len(gadm)

255

In [20]:
gadm.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'geometry'],
      dtype='object')

In [26]:
gadm.to_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/gadm0_precalculated_20220224_updated_NAME.geojson',driver='GeoJSON')

In [23]:
pwd

'/Users/sofia/Documents/Repos/he-scratchfolder'